# Palmer Penguins Modeling

Import the Palmer Penguins dataset and print out the first few rows.

Suppose we want to predict `bill_depth_mm` using the other variables in the dataset.

**Dummify** all variables that require this.

In [2]:
import numpy as np
import pandas as pd
from palmerpenguins import load_penguins
np.random.seed(42)
penguins = load_penguins()
target = "bill_depth_mm"
num_cols = ["bill_length_mm", "flipper_length_mm", "body_mass_g", "year"]
cat_cols = ["species", "island", "sex"]
need_cols = [target] + num_cols + cat_cols
dat = penguins[need_cols].dropna().copy()
dat_dum = pd.get_dummies(dat, columns=cat_cols, drop_first=True)
y = dat_dum[target]
X_full = dat_dum.drop(columns=[target])
X_full.head()

,bill_length_mm,flipper_length_mm,body_mass_g,year,species_Chinstrap,species_Gentoo,island_Dream,island_Torgersen,sex_male
0,39.1,181.0,3750.0,2007,False,False,False,True,True
1,39.5,186.0,3800.0,2007,False,False,False,True,False
2,40.3,195.0,3250.0,2007,False,False,False,True,False
4,36.7,193.0,3450.0,2007,False,False,False,True,False
5,39.3,190.0,3650.0,2007,False,False,False,True,True


Let's use the other variables to predict `bill_depth_mm`. Prepare your data and fit the following models on a training dataset subset of the entire dataset:

* Four different models, each containing a different set of predictor variables

Create a plot like the right plot of Fig 1. in our `Model Validation` chapter with the training and test error plotted for each of your four models.

Which of your models was best?

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import math
X_train, X_test, y_train, y_test = train_test_split(
    X_full, y, test_size=0.25, random_state=42
)
species_cols = [c for c in X_full.columns if c.startswith("species_")]
island_cols  = [c for c in X_full.columns if c.startswith("island_")]
sex_cols     = [c for c in X_full.columns if c.startswith("sex_")]
models = {
    "M1_len_only"    : ["bill_length_mm"],
    "M2_numeric"     : ["bill_length_mm", "flipper_length_mm", "body_mass_g", "year"],
    "M3_num+species" : ["bill_length_mm", "flipper_length_mm", "body_mass_g", "year"] + species_cols,
    "M4_full"        : ["bill_length_mm", "flipper_length_mm", "body_mass_g", "year"] + species_cols + island_cols + sex_cols,
}
models = {
    name: [c for c in cols if c in X_train.columns and c in X_test.columns]
    for name, cols in models.items()
}
rows = []
for name, cols in models.items():
    lr = LinearRegression()
    lr.fit(X_train[cols], y_train)
    yhat_tr = lr.predict(X_train[cols])
    yhat_te = lr.predict(X_test[cols])
    rmse_tr = math.sqrt(mean_squared_error(y_train, yhat_tr))
    rmse_te = math.sqrt(mean_squared_error(y_test,  yhat_te))
    rows.append({"model": name, "split": "Train", "rmse": rmse_tr, "k": len(cols)})
    rows.append({"model": name, "split": "Test",  "rmse": rmse_te, "k": len(cols)})
metrics_df = pd.DataFrame(rows)
from plotnine import ggplot, aes, geom_point, geom_line, theme, element_text, labs
metrics_df = metrics_df.sort_values(["k","model"]).copy()
metrics_df["model_order"] = pd.Categorical(metrics_df["model"],
                                           categories=metrics_df["model"].unique(), ordered=True)
p = (
    ggplot(metrics_df, aes(x="model_order", y="rmse", group="split", color="split"))
    + geom_line()
    + geom_point(size=3)
    + labs(
        title="Training vs Test RMSE by Model (Palmer Penguins, target = bill_depth_mm)",
        x="Model (increasing complexity →)",
        y="RMSE",
        color="Data split"
    )
    + theme(axis_text_x=element_text(rotation=30, hjust=1))
)
p
best_row = metrics_df[metrics_df["split"]=="Test"].sort_values("rmse").iloc[0]
print(f"Best model by Test RMSE: {best_row['model']}  (RMSE = {best_row['rmse']:.3f}, predictors = {int(best_row['k'])})")

Best model by Test RMSE: M4_full  (RMSE = 0.901, predictors = 9)


Best model: M4_full (Test RMSE = 0.901, 9 predictors). Because it achieves the lowest Test RMSE, so it generalizes best among the four specs

Train vs. Test pattern: As model complexity increased, train error decreased while test error hit its minimum at M4